# Module 3: SPARQL queries 

**Level: Advanced (300)**  

 

Learning objectives:  

- The basic syntax of a SPARQL query.  
- Being able to run SPARQL queries on the endpoint of heritage institutions.

 
## Introduction

SPARQL (pronounced “sparkle”) is the query language for the Semantic Web. The name of the standard is an acronym which stands for SPARQL Protocol and RDF Query Language. Together with RDF , it is one of the three core technologies associated the Semantic Web. The language can be used to query data in RDF.  

 
To be able to retrieve and analyze Linked Open Data, you need to know how to build SPARQL queries. This is what you will learn within this module by looking at several sample queries, which you can also run in Jupyter Notebooks. You will also learn how to formulate SPARQL queries on Wikidata using the Wiki query service. 

 
## RDF 

 
RDF statements, as you saw in the first lesson, consist of three components: subjects, predicates and objects. You can see an example of an RDF statement below: 

In [ ]:
PREFIX schema: <http://schema.org/> 

<https://data.rkd.nl/artists/56854> 
<schema:name> 
'Piet Mondriaan'

In this example, the base URI for schema.org is also replaced with a prefix. This was done to make the URI shorter and more manageable. 

The identifier that serves as the subject in this RDF statement (or triple) was assigned by the RKD (Netherlands Institute for Art History). The RDF triple gives information about one of the artists described in their digital archive. More specifically, it states that the entity specified by the identifier (<https://data.rkd.nl/artists/56854>) has a name, namely “Piet Mondriaan”. The subject and the object of RDF triples can either be an URI or literal values. In the example above, the URIs are given in angular brackets, and the literals are given in quotes.  

## SPARQL 

 Now imagine a situation in which you have received a URI without any further context, such as <https://data.rkd.nl/artists/32439>. The identifiers created for the artists in the RKD do not reveal any details about the people that are being identified. In this situation, you would want to know which artist is being identified. We can use SPARQL to find more information about such identifiers. The RDF triple (which reads like an affirmative statement) firstly needs to be transformed into a question. You can do this as follows.  

In [ ]:
PREFIX rkdo: <http://data.rkd.nl/def#> 
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#> 
PREFIX schema: <http://schema.org/> 

SELECT ?name 
WHERE { 
<https://data.rkd.nl/artists/32439>  
schema:name  
?name  
} 

SPARQL queries look quite similar to regular RDF triples. A significant difference, however, is that certain components are replaced with variables. If you compare the query contained within the curly brackets following WHERE to the full RDF statement that was discussed earlier, you can see that the literal value that was used as a predicate (i.e., the actual name associated with the subject) has been replaced with a variable named ?name.  

You are free to choose the name of the variables yourself. In the SPARQL language, variables always start with a question mark. It is advisable to work with meaningful names, such as ?place for places or ?date for dates. 

## SELECT and WHERE 

To change a sentence into an actual SPARQL query, you also need to add the keywords SELECT and WHERE. The curly brackets that follow WHERE, contain one or more ‘incomplete’ RDF statements, or statements in which one of the central components (subject, object, and predicate) are replaced with variables. In the example above, the request is to ‘fill in the blanks’, or, in other words, to find the correct value for the variable. 

In the SELECT clause, you specify the variables whose values you would like to see. In this example there is only one variable, namely, ?name. The SPARQL query will return a table. The number of columns will be the same as the number of variables you mention after SELECT. Each value that can be found to complete one of the RDF statements in the WHERE clause will be displayed will generate a new row in this table.  

If you run the query above in the RKD SPARQL endpoint, available at https://data.netwerkdigitaalerfgoed.nl/rkd/rkdartists/sparql/rkdartists, the SPARQL engine will then try to complete the query. The values that are found are assigned to the variables. If everything goes well, you will see that the identifier that was supplied is associated with Vincent van Gogh.  

This is the basis of working with SPARQL.  

## Multiple variables 

If you want to see more details for a specific identifier, you can simply add more RDF statements containing variables in the WHERE clause. According to the rules of the SPARQL syntax, each individual RDF triple needs to end in a full stop.  

In [ ]:
PREFIX rkdo: <http://data.rkd.nl/def#> 
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#> 
PREFIX schema: <http://schema.org/> 

SELECT ?name ?gender ?date_birth 

WHERE { 
<https://data.rkd.nl/artists/32439> schema:name ?name . 
<https://data.rkd.nl/artists/32439> schema:gender ?gender . 
<https://data.rkd.nl/artists/32439> rkdo:Birth ?birth . 
?birth schema:startDate ?date_birth . 
} 

This query requests the name, the gender and the birth date of the artist identified by the URI that is mentioned. The variable ?date_birth is slightly confusing at first sight. The property rkdo:Birth initially returns a rkdo:Birth resource. This is a general resource that can describe the details of a specific birth. Such a birth event can be described using a schema:startDate property. To find the actual birth date of Vincent van Gogh in the RKD database, you need to work with these two separate queries.  

 
## LIMIT 

Following the principles that have been discussed so far, you can start to build more complicated queries, or queries that produce longer result lists.  

The query below, for example, requests the name and the gender of all the artists that are described in the RKD database. It firstly creates a variable named ?id, which represents all the identifiers that are assigned to people. In this example, the SPARQL endpoint will be able to find many potential matches for the ?id variable. For each of the identifiers, it tries to obtain the name and the gender. 

In [ ]:
PREFIX rkdo: <http://data.rkd.nl/def#> 
PREFIX schema: <http://schema.org/> 

SELECT ?id ?name ?gender ?date_birth ?date_death 

WHERE { 
?id a schema:Person . 
?id schema:name ?name . 
?id schema:gender ?gender. } 
LIMIT 100 

As you can imagine, this query would return an extensive list of results. When the result consists of several hundreds of items, it may take some time to load. SPARQL endpoints may also produce time-out errors in the case of such long lists of results. To avoid such errors, we can work with the LIMIT keyword, which needs to be followed by a number. Adding such a LIMIT section will have the fact that the length of the result list will not exceed the number that is specified.  

 
## FILTER 

You can add a FILTER clause to add some criteria for the results to be displayed. If you do not want to see all the artists, but only those artists that were born in a specific decade (in between 1890 and 1900, for example), you can filter the original result set. The FILTER keyword is followed by a set or parenthesis. Within these parentheses you can add Boolean expressions which define criteria for the variables you work with. To see the artists who were born in the last decade of the 19th century, you can work with the query below.  

 

In [ ]:
PREFIX rkdo: <http://data.rkd.nl/def#> 
PREFIX schema: <http://schema.org/> 

SELECT ?id ?name ?gender ?date_birth  

WHERE { 
?id a schema:Person . 
?id schema:name ?name . 
?id schema:gender ?gender . 
?id rkdo:Birth ?birth . 
?birth schema:startDate ?date_birth .   

FILTER ( ?date_birth >= "1890-01-01"^^xsd:date && ?date_birth < "1900-01-01"^^xsd:date) 

} 

## ORDER 

 
The ORDER keyword can be used to arrange the query results. ORDER needs to be included in the query after the closing bracket following WHERE. After ORDER, you can refer to any of the variables you have defined.  

In [ ]:
PREFIX rkdo: <http://data.rkd.nl/def#>  
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#> 
PREFIX schema: <http://schema.org/>  

SELECT ?id ?name ?gender ?date_birth     

WHERE {  
?id a schema:Person .  
?id schema:name ?name .  
?id schema:gender ?gender .  
?id rkdo:Birth ?birth .  
?birth schema:startDate ?date_birth .    

FILTER ( ?date_birth >= "1890-01-01"^^xsd:date && ?date_birth < "1900-01-01"^^xsd:date)  

} 

ORDER BY ?name  

## Wikidata 

Wikidata offers access to many of the facts that are listed on Wikipedia as RDF triplets. In other words, Wikidata presents the contents of Wikipedia as structured data. WikiData’s SPARQL Endpoint can be found at https://query.wikidata.org/sparql  

Wikidata does not make use of human-understandable labels. Objects, subjects and predicated are all represented as numeric codes, following either the letter ‘P’ or ‘W’. Named entities such as people, places, organizations and works of art have mostly been assigned a code starting with the letter ‘Q’. The properties all start with a ‘P’.  

You can see an example below.  

In [ ]:
SELECT ?work  

WHERE {  
?work wdt:P170 wd:Q151803 . 
} 

The property wdt:P170 was created to describe a “maker of this creative work or other object”. wd:Q151803 is the identifier that was assigned to Piet Mondriaan. The query shown in this example requests all the works of art created by the painter Piet Mondriaan. As you can see, the subject is represented as a variable named ?work.  

When you run the code in Wikidata’s SPARQL endpoint, the result will initially consist of numbers only, so this may look slightly disappointing. The identifiers, consisting of random codes, are not highly informative in themselves. Wikidata offers a useful service, however, which you can use to generate human-readable labels for the codes assigned by Wikidata.  

First, you need to add the following line to the query.  

 

In [ ]:
SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . } 

After wikibase:language, you can specify the language of the label. Next to “en”, which stands for English, you can also request labels in Dutch (“nl”), German (‘de”’) or French (“fr”). 

Once you have added this specific SERVICE clause, Wikidata creates a second version of all the variables you specify in the WHERE section of the query. The name of this second version of the query will be the name you assigned yourself, with the suffix ‘Label’ as an addition. If you created a variable named ?place, for example, the Wikibase Label service will create a ?placeLabel variable. This variable will contain a textual label associated with the Wikidata identifier.  

 

In [ ]:
SELECT ?work  ?workLabel 

WHERE  
{   
?work wdt:P170 wd:Q151803 .  
SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }  
} 

## Wiki query services 

Since 2015, Wikidata has provided services that can help you build SPARQL queries. One service is called the **query builder**. It can be accessed via https://query.wikidata.org/querybuilder/. This tool offers a visual interface that can be used to build simple SPARQL queries. Although that this service is limited in making SPARQL queries, it can be helpful if you do not have a lot of experience yet with building SPARQL queries. 

Another service offered by WikiData is the **query service** accessible via https://query.wikidata.org/. The Wikidata Query Service offers opportunities for building SPARQL queries and downloading the requested data in various file formats (including JSON and CSV). It is also possible to view the data in different data visualizations, like a table, a bar chart or a timeline.  


# Exercises
 
## Exercise 3.1.

Use the Wiki Query Builder to find artworks made by Leonardo da Vinci created after 1495. Run the query and look at the results 

Above the result (at the right top), there is a link to the Query Service. Open this query within the Wiki Query Service, 

## Exercise 3.2.
 
Use the query that you have built in the previous exercise (artworks made by Leonardo da Vinci created after 1495) and show it in the Wiki Query Service. 

Change the year into 1505 and the artist in Michelangelo. The standard limit is 100 items, set the limit to 1000. 

Go to Download at the top right of the results screen and download a CSV file.  
